<a href="https://colab.research.google.com/github/dsfernandez/DoE-RENEW/blob/main/renew_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing act-atmos
Following README.rst in https://github.com/ARM-DOE/ACT:

ACT can be installed a few different ways. One way is to install using pip. When installing with pip, the ACT dependencies found in requirements.txt will also be installed. To install using pip:

In [ ]:
!pip install act-atmos